# 02.1 Overall Health by State - Wrangling

1. Imports and display options
1. General Health
1. Diet
1. Mental Health
1. Physical Activity
1. Overall Health
1. Exports

## 1. Imports and display options

In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
#Creating a path to the root directory for this project
path = r'C:\Users\theva\OneDrive\Documents\Data Analyses\BRFSS'

In [3]:
#Importing products.csv to df_prods
df = pd.read_pickle(os.path.join(path, '02 Data','Prepared Data', 'BRFSS CLEANED.pkl'))

In [4]:
#Checking df

df.shape

(727716, 10)

In [5]:
#Checking df

df.head()

,year,sample_size,data_value,class_id,topic_id,location_id,breakout_id,breakout_category_id,question_id,response_id
11,2019,1235,76.5,CLASS07,TOPIC36,20,AGE04,CAT3,CHECKUP1,RESP063
18,2020,34,2.0,CLASS05,TOPIC48,1,EDUCA3,CAT5,RACE8,RESP200
19,2020,401,47.5,CLASS05,TOPIC51,1,AGE05,CAT3,SEX,RESP001
26,2019,52,4.3,CLASS17,TOPIC54,20,AGE02,CAT3,USENOW3,RESP079
28,2019,17,0.4,CLASS17,TOPIC54,20,SEX2,CAT2,USENOW3,RESP080


In [6]:
#Expanding max window size

pd.set_option('display.max_rows', 200)
pd.set_option('display.max_colwidth', None)

# 2. General Health

In [7]:
# Making a dataframe for overall (BO1)  BMI (_BMI5CAT), general health (GENHLTH), health status (_RFHLTH),
# and last checkup (CHECKUP1) in all locations and years

gh_base = df.loc[((df['question_id'] == '_BMI5CAT') |
                 (df['question_id'] == 'GENHLTH') | 
                 (df['question_id'] == '_RFHLTH') | 
                 (df['question_id'] == 'CHECKUP1')) &
                 (df['breakout_id'] == 'BO1')][['year', 'location_id', 'sample_size', 'data_value', 'question_id', 'response_id']]
gh_base

,year,location_id,sample_size,data_value,question_id,response_id
74,2019,21,2706,35.4,_BMI5CAT,RESP040
180,2020,5,27,0.8,CHECKUP1,RESP067
221,2019,2,859,32.2,_BMI5CAT,RESP041
244,2020,6,23,0.7,CHECKUP1,RESP067
370,2020,9,22,0.3,CHECKUP1,RESP067
...,...,...,...,...,...,...
2289292,2021,98,50,3.6,GENHLTH,RESP060
2289553,2021,72,926,24.7,GENHLTH,RESP056
2289637,2021,72,51,1.6,_BMI5CAT,RESP042
2289639,2021,78,1147,78.6,_RFHLTH,RESP061


In [8]:
#Starting general health dataframe

gh = pd.DataFrame(gh_base.groupby(['location_id', 'year'])[['location_id','year']].min()).reset_index(drop=True)
gh

,location_id,year
0,1,2015
1,1,2016
2,1,2017
3,1,2018
4,1,2019
...,...,...
380,99,2017
381,99,2018
382,99,2019
383,99,2020


In [9]:
#Checking response values for these questions

gh_base.groupby(['question_id', 'response_id']).count()

year  location_id  sample_size  data_value
question_id response_id                                            
CHECKUP1    RESP063       385          385          385         385
            RESP064       385          385          385         385
            RESP065       385          385          385         385
            RESP066       385          385          385         384
            RESP067       385          385          385         327
GENHLTH     RESP056       385          385          385         385
            RESP057       385          385          385         385
            RESP058       385          385          385         385
            RESP059       385          385          385         385
            RESP060       385          385          385         385
_BMI5CAT    RESP039       385          385          385         385
            RESP040       385          385          385         385
            RESP041       385          385          385         385
            RESP042       385          385          385         381
_RFHLTH     RESP061       385          385          385         385
            RESP062       385          385          385         385

### BMI (_BMI5CAT)

In [10]:
#making key dataframes for each response to this question

r039 = gh_base.loc[gh_base['response_id'] == 'RESP039'][['location_id', 'year', 'data_value', 'sample_size']]
r039.rename(columns = {'data_value' : 'resp039_value',
                       'sample_size' : 'resp039_sample'}, inplace = True)

r040 = gh_base.loc[gh_base['response_id'] == 'RESP040'][['location_id', 'year', 'data_value', 'sample_size']]
r040.rename(columns = {'data_value' : 'resp040_value',
                       'sample_size' : 'resp040_sample'}, inplace = True)

r041 = gh_base.loc[gh_base['response_id'] == 'RESP041'][['location_id', 'year', 'data_value', 'sample_size']]
r041.rename(columns = {'data_value' : 'resp041_value',
                       'sample_size' : 'resp041_sample'}, inplace = True)

r042 = gh_base.loc[gh_base['response_id'] == 'RESP042'][['location_id', 'year', 'data_value', 'sample_size']]
r042.rename(columns = {'data_value' : 'resp042_value',
                       'sample_size' : 'resp042_sample'}, inplace = True)

In [11]:
#Merging gh with keys

gh = pd.merge(gh, r039, how='left', left_on=['location_id', 'year'], right_on=['location_id', 'year'])
gh = pd.merge(gh, r040, how='left', left_on=['location_id', 'year'], right_on=['location_id', 'year'])
gh = pd.merge(gh, r041, how='left', left_on=['location_id', 'year'], right_on=['location_id', 'year'])
gh = pd.merge(gh, r042, how='left', left_on=['location_id', 'year'], right_on=['location_id', 'year'])

### General Health (GENHLTH)

In [12]:
#making key dataframes for each response to this question

r056 = gh_base.loc[gh_base['response_id'] == 'RESP056'][['location_id', 'year', 'data_value', 'sample_size']]
r056.rename(columns = {'data_value' : 'resp056_value',
                       'sample_size' : 'resp056_sample'}, inplace = True)

r057 = gh_base.loc[gh_base['response_id'] == 'RESP057'][['location_id', 'year', 'data_value', 'sample_size']]
r057.rename(columns = {'data_value' : 'resp057_value',
                       'sample_size' : 'resp057_sample'}, inplace = True)

r058 = gh_base.loc[gh_base['response_id'] == 'RESP058'][['location_id', 'year', 'data_value', 'sample_size']]
r058.rename(columns = {'data_value' : 'resp058_value',
                       'sample_size' : 'resp058_sample'}, inplace = True)

r059 = gh_base.loc[gh_base['response_id'] == 'RESP059'][['location_id', 'year', 'data_value', 'sample_size']]
r059.rename(columns = {'data_value' : 'resp059_value',
                       'sample_size' : 'resp059_sample'}, inplace = True)

r060 = gh_base.loc[gh_base['response_id'] == 'RESP060'][['location_id', 'year', 'data_value', 'sample_size']]
r060.rename(columns = {'data_value' : 'resp060_value',
                       'sample_size' : 'resp060_sample'}, inplace = True)

In [13]:
#Merging gh with keys

gh = pd.merge(gh, r056, how='left', left_on=['location_id', 'year'], right_on=['location_id', 'year'])
gh = pd.merge(gh, r057, how='left', left_on=['location_id', 'year'], right_on=['location_id', 'year'])
gh = pd.merge(gh, r058, how='left', left_on=['location_id', 'year'], right_on=['location_id', 'year'])
gh = pd.merge(gh, r059, how='left', left_on=['location_id', 'year'], right_on=['location_id', 'year'])
gh = pd.merge(gh, r060, how='left', left_on=['location_id', 'year'], right_on=['location_id', 'year'])

### Health Status (_RFHLTH)

In [14]:
#making key dataframes for each response to this question

r061 = gh_base.loc[gh_base['response_id'] == 'RESP061'][['location_id', 'year', 'data_value', 'sample_size']]
r061.rename(columns = {'data_value' : 'resp061_value',
                       'sample_size' : 'resp061_sample'}, inplace = True)

r062 = gh_base.loc[gh_base['response_id'] == 'RESP062'][['location_id', 'year', 'data_value', 'sample_size']]
r062.rename(columns = {'data_value' : 'resp062_value',
                       'sample_size' : 'resp062_sample'}, inplace = True)

In [15]:
#Merging gh with keys

gh = pd.merge(gh, r061, how='left', left_on=['location_id', 'year'], right_on=['location_id', 'year'])
gh = pd.merge(gh, r062, how='left', left_on=['location_id', 'year'], right_on=['location_id', 'year'])

### Last Checkup (CHECKUP1)

In [16]:
#making key dataframes for each response to this question

r063 = gh_base.loc[gh_base['response_id'] == 'RESP063'][['location_id', 'year', 'data_value', 'sample_size']]
r063.rename(columns = {'data_value' : 'resp063_value',
                       'sample_size' : 'resp063_sample'}, inplace = True)

r064 = gh_base.loc[gh_base['response_id'] == 'RESP064'][['location_id', 'year', 'data_value', 'sample_size']]
r064.rename(columns = {'data_value' : 'resp064_value',
                       'sample_size' : 'resp064_sample'}, inplace = True)

r065 = gh_base.loc[gh_base['response_id'] == 'RESP065'][['location_id', 'year', 'data_value', 'sample_size']]
r065.rename(columns = {'data_value' : 'resp065_value',
                       'sample_size' : 'resp065_sample'}, inplace = True)

r066 = gh_base.loc[gh_base['response_id'] == 'RESP066'][['location_id', 'year', 'data_value', 'sample_size']]
r066.rename(columns = {'data_value' : 'resp066_value',
                       'sample_size' : 'resp066_sample'}, inplace = True)

r067 = gh_base.loc[gh_base['response_id'] == 'RESP067'][['location_id', 'year', 'data_value', 'sample_size']]
r067.rename(columns = {'data_value' : 'resp067_value',
                       'sample_size' : 'resp067_sample'}, inplace = True)

In [17]:
#Merging gh with keys

gh = pd.merge(gh, r063, how='left', left_on=['location_id', 'year'], right_on=['location_id', 'year'])
gh = pd.merge(gh, r064, how='left', left_on=['location_id', 'year'], right_on=['location_id', 'year'])
gh = pd.merge(gh, r065, how='left', left_on=['location_id', 'year'], right_on=['location_id', 'year'])
gh = pd.merge(gh, r066, how='left', left_on=['location_id', 'year'], right_on=['location_id', 'year'])
gh = pd.merge(gh, r067, how='left', left_on=['location_id', 'year'], right_on=['location_id', 'year'])

In [18]:
#Checking general health dataframe

gh.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 385 entries, 0 to 384
Data columns (total 34 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   location_id     385 non-null    object 
 1   year            385 non-null    int64  
 2   resp039_value   385 non-null    float64
 3   resp039_sample  385 non-null    int64  
 4   resp040_value   385 non-null    float64
 5   resp040_sample  385 non-null    int64  
 6   resp041_value   385 non-null    float64
 7   resp041_sample  385 non-null    int64  
 8   resp042_value   381 non-null    float64
 9   resp042_sample  385 non-null    int64  
 10  resp056_value   385 non-null    float64
 11  resp056_sample  385 non-null    int64  
 12  resp057_value   385 non-null    float64
 13  resp057_sample  385 non-null    int64  
 14  resp058_value   385 non-null    float64
 15  resp058_sample  385 non-null    int64  
 16  resp059_value   385 non-null    float64
 17  resp059_sample  385 non-null    int

In [19]:
#Checking general health dataframe

gh.head(200)

,location_id,year,resp039_value,resp039_sample,resp040_value,resp040_sample,resp041_value,resp041_sample,resp042_value,resp042_sample,...,resp063_value,resp063_sample,resp064_value,resp064_sample,resp065_value,resp065_sample,resp066_value,resp066_sample,resp067_value,resp067_sample
0,1,2015,35.6,2639,33.0,2523,28.9,2075,2.4,146,...,71.7,6123,12.0,751,6.5,400,8.4,480,1.3,77
1,1,2016,35.7,2352,33.8,2259,28.4,1805,2.1,110,...,72.5,5392,11.3,660,7.0,379,8.1,427,1.1,55
2,1,2017,36.3,2260,33.9,2152,28.5,1748,1.3,85,...,71.9,5176,11.8,677,6.5,333,8.7,431,1.2,48
3,1,2018,36.2,2238,33.4,2169,28.4,1633,2.0,104,...,77.2,5471,11.6,574,4.9,212,5.8,252,0.5,24
4,1,2019,36.1,2338,34.2,2295,27.5,1758,2.2,130,...,78.5,5841,9.7,580,5.2,267,5.9,280,0.7,27
5,1,2020,39.0,1918,33.7,1695,25.4,1289,1.8,71,...,76.5,4286,10.4,484,6.3,251,5.8,240,0.9,34
6,1,2021,39.9,1652,31.9,1412,26.4,1112,1.8,71,...,78.0,3732,11.0,459,4.9,170,5.4,167,0.6,17
7,10,2015,29.7,1146,37.2,1391,31.7,1092,1.4,59,...,75.6,3216,11.9,380,6.2,193,5.0,169,1.2,42
8,10,2016,30.7,1180,37.3,1407,30.2,1063,1.8,52,...,78.2,3257,10.4,361,5.5,178,5.3,174,0.7,34
9,10,2017,31.8,1224,36.7,1329,29.5,1039,2.0,56,...,78.3,3236,11.6,418,4.3,181,4.9,192,0.9,47


In [20]:
#Checking general health dataframe

gh.tail(200)

,location_id,year,resp039_value,resp039_sample,resp040_value,resp040_sample,resp041_value,resp041_sample,resp042_value,resp042_sample,...,resp063_value,resp063_sample,resp064_value,resp064_sample,resp065_value,resp065_sample,resp066_value,resp066_sample,resp067_value,resp067_sample
185,35,2020,30.9,1879,36.9,2345,29.9,2113,2.4,148,...,69.6,5124,16.0,1017,6.8,417,6.8,342,0.8,48
186,35,2021,34.6,1972,35.4,2094,28.0,1729,2.0,112,...,63.0,4221,19.2,1144,9.1,507,7.5,394,1.1,56
187,36,2015,25.0,2940,34.5,4011,38.4,4121,2.1,209,...,72.3,9461,13.3,1395,8.2,771,5.2,548,1.0,71
188,36,2016,25.5,9469,35.3,11215,37.0,10044,2.2,541,...,74.9,26480,12.4,3508,6.9,2002,5.0,1777,0.8,178
189,36,2017,25.7,2994,35.6,4106,36.5,3871,2.2,209,...,74.6,9321,12.5,1399,7.3,782,4.5,537,1.0,90
190,36,2018,27.6,9963,35.1,11317,34.9,10171,2.4,625,...,81.8,29637,10.2,3064,4.4,1357,3.2,1223,0.4,129
191,36,2019,27.1,3680,36.1,4573,35.0,4142,1.8,236,...,81.0,11914,10.0,1151,5.1,570,3.3,410,0.6,56
192,36,2020,26.3,3422,37.0,4668,34.2,4231,2.4,262,...,79.7,11944,12.7,1654,4.6,595,2.5,363,0.5,56
193,36,2021,29.1,10857,34.5,11751,34.1,9903,2.2,610,...,78.9,31717,12.8,4018,4.9,1551,2.7,1129,0.7,168
194,37,2015,30.1,1808,35.8,2214,32.7,1903,1.5,94,...,73.5,5020,12.6,769,7.3,419,5.6,345,1.0,61


# 3. Diet

In [21]:
#Making a dataframe for overall (BO1) fruit (_FRTLT1A) and vegetable (_VEGLT1A) consumption in all locations and years

diet_base = df.loc[((df['question_id'] == '_FRTLT1A') |
                    (df['question_id'] == '_VEGLT1A')) &
                   (df['breakout_id'] == 'BO1')][['year', 'location_id', 'sample_size', 'data_value', 'question_id', 'response_id']]
diet_base

,year,location_id,sample_size,data_value,question_id,response_id
2592,2019,56,2698,57.8,_FRTLT1A,RESP077
3029,2019,4,2945,38.3,_FRTLT1A,RESP078
6027,2019,2,471,19.0,_VEGLT1A,RESP078
7253,2019,2,1577,57.4,_FRTLT1A,RESP077
8505,2019,2,2120,81.0,_VEGLT1A,RESP077
...,...,...,...,...,...,...
2288997,2021,98,50,80.3,_VEGLT1A,RESP077
2289032,2021,72,1903,51.1,_FRTLT1A,RESP078
2289102,2021,78,399,39.7,_FRTLT1A,RESP078
2289614,2021,99,53,41.0,_FRTLT1A,RESP078


In [22]:
#Starting diet dataframe

diet = pd.DataFrame(diet_base.groupby(['location_id', 'year'])[['location_id','year']].min()).reset_index(drop=True)
diet

,location_id,year
0,1,2015
1,1,2017
2,1,2019
3,1,2021
4,10,2015
...,...,...
214,98,2021
215,99,2015
216,99,2017
217,99,2019


In [23]:
#Checking response values for these questions

diet_base.groupby(['question_id', 'response_id']).count()

year  location_id  sample_size  data_value
question_id response_id                                            
_FRTLT1A    RESP077       219          219          219         219
            RESP078       219          219          219         219
_VEGLT1A    RESP077       219          219          219         219
            RESP078       219          219          219         219

In [24]:
#making key dataframes for each question/response

f077 = diet_base.loc[(diet_base['question_id'] == '_FRTLT1A') &
                     (diet_base['response_id'] == 'RESP077')][['location_id', 'year', 'data_value', 'sample_size']]
f077.rename(columns = {'data_value' : 'fruit_resp077_value', 'sample_size' : 'fruit_resp077_sample'}, inplace = True)

f078 = diet_base.loc[(diet_base['question_id'] == '_FRTLT1A') &
                     (diet_base['response_id'] == 'RESP078')][['location_id', 'year', 'data_value', 'sample_size']]
f078.rename(columns = {'data_value' : 'fruit_resp078_value', 'sample_size' : 'fruit_resp078_sample'}, inplace = True)

v077 = diet_base.loc[(diet_base['question_id'] == '_VEGLT1A') &
                     (diet_base['response_id'] == 'RESP077')][['location_id', 'year', 'data_value', 'sample_size']]
v077.rename(columns = {'data_value' : 'veg_resp077_value', 'sample_size' : 'veg_resp077_sample'}, inplace = True)

v078 = diet_base.loc[(diet_base['question_id'] == '_VEGLT1A') &
                     (diet_base['response_id'] == 'RESP078')][['location_id', 'year', 'data_value', 'sample_size']]
v078.rename(columns = {'data_value' : 'veg_resp078_value', 'sample_size' : 'veg_resp078_sample'}, inplace = True)

In [25]:
#Merging diet with keys

diet = pd.merge(diet, f077, how='left', left_on=['location_id', 'year'], right_on=['location_id', 'year'])
diet = pd.merge(diet, f078, how='left', left_on=['location_id', 'year'], right_on=['location_id', 'year'])
diet = pd.merge(diet, v077, how='left', left_on=['location_id', 'year'], right_on=['location_id', 'year'])
diet = pd.merge(diet, v078, how='left', left_on=['location_id', 'year'], right_on=['location_id', 'year'])

In [26]:
#Checking diet dataframe

diet.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 219 entries, 0 to 218
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   location_id           219 non-null    object 
 1   year                  219 non-null    int64  
 2   fruit_resp077_value   219 non-null    float64
 3   fruit_resp077_sample  219 non-null    int64  
 4   fruit_resp078_value   219 non-null    float64
 5   fruit_resp078_sample  219 non-null    int64  
 6   veg_resp077_value     219 non-null    float64
 7   veg_resp077_sample    219 non-null    int64  
 8   veg_resp078_value     219 non-null    float64
 9   veg_resp078_sample    219 non-null    int64  
dtypes: float64(4), int64(5), object(1)
memory usage: 18.8+ KB


In [27]:
#checking diet dataframe

diet.head(110)

,location_id,year,fruit_resp077_value,fruit_resp077_sample,fruit_resp078_value,fruit_resp078_sample,veg_resp077_value,veg_resp077_sample,veg_resp078_value,veg_resp078_sample
0,1,2015,51.7,3791,48.3,3333,72.2,5130,27.8,1808
1,1,2017,55.2,3523,44.8,2618,80.7,4878,19.3,1168
2,1,2019,53.9,3476,46.1,2802,77.7,4770,22.3,1336
3,1,2021,54.2,2337,45.8,1845,79.4,3315,20.6,805
4,10,2015,61.8,2350,38.2,1320,79.3,2852,20.7,719
5,10,2017,64.6,2434,35.4,1254,82.8,2978,17.2,612
6,10,2019,63.0,2167,37.0,1223,79.7,2670,20.3,635
7,10,2021,60.3,1931,39.7,1215,79.5,2483,20.5,554
8,11,2015,63.0,2454,37.0,1020,81.0,2828,19.0,586
9,11,2017,68.9,2497,31.1,992,87.7,2924,12.3,419


In [28]:
#checking diet dataframe

diet.tail(110)

,location_id,year,fruit_resp077_value,fruit_resp077_sample,fruit_resp078_value,fruit_resp078_sample,veg_resp077_value,veg_resp077_sample,veg_resp078_value,veg_resp078_sample
109,36,2021,64.3,21800,35.7,11500,79.3,26384,20.7,5847
110,37,2015,56.7,3597,43.3,2488,78.4,4781,21.6,1178
111,37,2017,62.6,2885,37.4,1612,84.4,3737,15.6,646
112,37,2019,61.9,2455,38.1,1429,82.9,3134,17.1,647
113,37,2021,59.7,2796,40.3,1808,82.0,3728,18.0,802
114,38,2015,59.6,2931,40.4,1648,72.5,3390,27.5,1129
115,38,2017,63.2,4277,36.8,2264,80.2,5301,19.8,1149
116,38,2019,59.4,3377,40.6,1811,77.9,4064,22.1,1042
117,38,2021,56.0,3220,44.0,2156,77.3,4180,22.7,1098
118,39,2015,57.1,6477,42.9,4257,75.2,7980,24.8,2529


# 4. Mental Health

In [29]:
#Making a dataframe for overall (BO1) mental health days (_MENT14D) and mental difficulty (DECIDE)


mh_base = df.loc[((df['question_id'] == '_MENT14D') |
                    (df['question_id'] == 'DECIDE')) &
                   (df['breakout_id'] == 'BO1')][['year', 'location_id', 'sample_size', 'data_value', 'question_id', 'response_id']]
mh_base

,year,location_id,sample_size,data_value,question_id,response_id
104,2020,4,1005,9.9,DECIDE,RESP046
882,2020,18,5298,60.0,_MENT14D,RESP222
1500,2019,28,749,17.3,_MENT14D,RESP224
2394,2019,54,914,19.2,DECIDE,RESP046
3184,2019,4,5652,60.0,_MENT14D,RESP222
...,...,...,...,...,...,...
2289049,2021,99,53,14.6,_MENT14D,RESP224
2289127,2021,72,626,16.8,DECIDE,RESP046
2289174,2021,78,1198,85.6,DECIDE,RESP054
2289448,2021,99,53,87.8,DECIDE,RESP054


In [30]:
#Starting transposed diet dataframe

mh = pd.DataFrame(mh_base.groupby(['location_id', 'year'])[['location_id','year']].min()).reset_index(drop=True)
mh

,location_id,year
0,1,2015
1,1,2016
2,1,2017
3,1,2018
4,1,2019
...,...,...
380,99,2017
381,99,2018
382,99,2019
383,99,2020


In [31]:
#Checking response values for these questions

mh_base.groupby(['question_id', 'response_id']).count()

year  location_id  sample_size  data_value
question_id response_id                                            
DECIDE      RESP046       385          385          385         385
            RESP054       385          385          385         385
_MENT14D    RESP222       164          164          164         164
            RESP223       164          164          164         164
            RESP224       164          164          164         164

In [32]:
#making key dataframes for each question/response

mh046 = mh_base.loc[(mh_base['question_id'] == 'DECIDE') &
                    (mh_base['response_id'] == 'RESP046')][['location_id', 'year', 'data_value', 'sample_size']]
mh046.rename(columns = {'data_value' : 'DECIDE_yes_value', 'sample_size' : 'DECIDE_yes_sample'}, inplace = True)

mh054 = mh_base.loc[(mh_base['question_id'] == 'DECIDE') &
                    (mh_base['response_id'] == 'RESP054')][['location_id', 'year', 'data_value', 'sample_size']]
mh054.rename(columns = {'data_value' : 'DECIDE_no_value', 'sample_size' : 'DECIDE_no_sample'}, inplace = True)

r222 = mh_base.loc[(mh_base['question_id'] == '_MENT14D') &
                   (mh_base['response_id'] == 'RESP222')][['location_id', 'year', 'data_value', 'sample_size']]
r222.rename(columns = {'data_value' : 'resp222_value', 'sample_size' : 'resp222_sample'}, inplace = True)

r223 = mh_base.loc[(mh_base['question_id'] == '_MENT14D') &
                   (mh_base['response_id'] == 'RESP223')][['location_id', 'year', 'data_value', 'sample_size']]
r223.rename(columns = {'data_value' : 'resp223_value', 'sample_size' : 'resp223_sample'}, inplace = True)

r224 = mh_base.loc[(mh_base['question_id'] == '_MENT14D') &
                   (mh_base['response_id'] == 'RESP224')][['location_id', 'year', 'data_value', 'sample_size']]
r224.rename(columns = {'data_value' : 'resp224_value', 'sample_size' : 'resp224_sample'}, inplace = True)

In [33]:
#Merging mh with keys

mh = pd.merge(mh, mh046, how='left', left_on=['location_id', 'year'], right_on=['location_id', 'year'])
mh = pd.merge(mh, mh054, how='left', left_on=['location_id', 'year'], right_on=['location_id', 'year'])
mh = pd.merge(mh, r222, how='left', left_on=['location_id', 'year'], right_on=['location_id', 'year'])
mh = pd.merge(mh, r223, how='left', left_on=['location_id', 'year'], right_on=['location_id', 'year'])
mh = pd.merge(mh, r224, how='left', left_on=['location_id', 'year'], right_on=['location_id', 'year'])

In [34]:
#Checking mental health dataframe

mh.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 385 entries, 0 to 384
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   location_id        385 non-null    object 
 1   year               385 non-null    int64  
 2   DECIDE_yes_value   385 non-null    float64
 3   DECIDE_yes_sample  385 non-null    int64  
 4   DECIDE_no_value    385 non-null    float64
 5   DECIDE_no_sample   385 non-null    int64  
 6   resp222_value      164 non-null    float64
 7   resp222_sample     164 non-null    float64
 8   resp223_value      164 non-null    float64
 9   resp223_sample     164 non-null    float64
 10  resp224_value      164 non-null    float64
 11  resp224_sample     164 non-null    float64
dtypes: float64(8), int64(3), object(1)
memory usage: 39.1+ KB


In [35]:
#checking mental health dataframe

mh.head(200)

,location_id,year,DECIDE_yes_value,DECIDE_yes_sample,DECIDE_no_value,DECIDE_no_sample,resp222_value,resp222_sample,resp223_value,resp223_sample,resp224_value,resp224_sample
0,1,2015,15.0,1174,85.0,6526,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2016,14.3,979,85.7,5812,NaN,NaN,NaN,NaN,NaN,NaN
2,1,2017,15.8,998,84.2,5498,NaN,NaN,NaN,NaN,NaN,NaN
3,1,2018,15.8,958,84.2,5423,NaN,NaN,NaN,NaN,NaN,NaN
4,1,2019,16.7,1016,83.3,5763,58.0,4302.0,24.5,1580.0,17.5,1040.0
5,1,2020,14.0,687,86.0,4465,60.8,3319.0,23.4,1177.0,15.8,757.0
6,1,2021,15.1,570,84.9,3853,56.7,2748.0,24.9,1061.0,18.4,705.0
7,10,2015,10.8,439,89.2,3519,NaN,NaN,NaN,NaN,NaN,NaN
8,10,2016,9.8,398,90.2,3542,NaN,NaN,NaN,NaN,NaN,NaN
9,10,2017,13.2,518,86.8,3478,NaN,NaN,NaN,NaN,NaN,NaN


In [36]:
#checking mental health dataframe

mh.tail(110)

,location_id,year,DECIDE_yes_value,DECIDE_yes_sample,DECIDE_no_value,DECIDE_no_sample,resp222_value,resp222_sample,resp223_value,resp223_sample,resp224_value,resp224_sample
275,48,2019,11.4,1365,88.6,10337,64.0,7929.0,23.8,2610.0,12.2,1459.0
276,48,2020,10.9,1267,89.1,9151,63.6,6948.0,23.1,2310.0,13.2,1501.0
277,48,2021,12.2,1261,87.8,8974,59.9,6428.0,26.1,2596.0,14.0,1515.0
278,49,2015,8.8,976,91.2,10116,NaN,NaN,NaN,NaN,NaN,NaN
279,49,2016,9.0,949,91.0,9703,NaN,NaN,NaN,NaN,NaN,NaN
280,49,2017,10.6,1028,89.4,8960,NaN,NaN,NaN,NaN,NaN,NaN
281,49,2018,10.9,1065,89.1,9279,NaN,NaN,NaN,NaN,NaN,NaN
282,49,2019,11.0,1214,89.0,10274,54.2,6840.0,31.6,3327.0,14.2,1435.0
283,49,2020,10.8,1063,89.2,9487,53.4,6251.0,31.6,3090.0,15.0,1394.0
284,49,2021,12.9,1168,87.1,8968,50.2,5809.0,32.8,3145.0,17.1,1488.0


# 5. Physical Activity

In [37]:
# Making a dataframe for overall (BO1) recent acticity (_TOTINDA), aerobic activity (_PAINDX2),
# ideal activity (_PASTAE2), and strength training (_PASTRNG) in all locations and years

pa_base = df.loc[((df['question_id'] == '_TOTINDA') |
                 (df['question_id'] == '_PAINDX2') | 
                 (df['question_id'] == '_PASTAE2') | 
                 (df['question_id'] == '_PASTRNG')) &
                 (df['breakout_id'] == 'BO1')][['year', 'location_id', 'sample_size', 'data_value', 'question_id', 'response_id']]
pa_base

,year,location_id,sample_size,data_value,question_id,response_id
76,2019,21,2622,32.8,_TOTINDA,RESP054
419,2019,6,8293,77.6,_TOTINDA,RESP046
1133,2020,23,2518,21.5,_TOTINDA,RESP054
1856,2020,34,9062,79.0,_TOTINDA,RESP046
1964,2019,36,2904,24.1,_PASTAE2,RESP046
...,...,...,...,...,...,...
2287776,2021,72,2218,53.9,_TOTINDA,RESP046
2288187,2021,66,1198,70.1,_TOTINDA,RESP046
2288277,2021,98,50,23.7,_TOTINDA,RESP054
2288780,2021,98,50,76.3,_TOTINDA,RESP046


In [38]:
#Starting physical activity dataframe

pa = pd.DataFrame(pa_base.groupby(['location_id', 'year'])[['location_id','year']].min()).reset_index(drop=True)
pa

,location_id,year
0,1,2015
1,1,2016
2,1,2017
3,1,2018
4,1,2019
...,...,...
380,99,2017
381,99,2018
382,99,2019
383,99,2020


In [39]:
#Checking response values for these questions

pa_base.groupby(['question_id', 'response_id']).count()

year  location_id  sample_size  data_value
question_id response_id                                            
_PAINDX2    RESP046       164          164          164         164
            RESP054       164          164          164         164
_PASTAE2    RESP046       164          164          164         164
            RESP054       164          164          164         164
_PASTRNG    RESP046       164          164          164         164
            RESP054       164          164          164         164
_TOTINDA    RESP046       385          385          385         385
            RESP054       385          385          385         385

In [40]:
#making key dataframes for each response to these questions

pa01 = pa_base.loc[(pa_base['question_id'] == '_PAINDX2') &
                    (pa_base['response_id'] == 'RESP046')][['location_id', 'year', 'data_value', 'sample_size']]
pa01.rename(columns = {'data_value' : 'PAINDX2_yes_value', 'sample_size' : 'PAINDX2_yes_sample'}, inplace = True)

pa02 = pa_base.loc[(pa_base['question_id'] == '_PAINDX2') &
                    (pa_base['response_id'] == 'RESP054')][['location_id', 'year', 'data_value', 'sample_size']]
pa02.rename(columns = {'data_value' : 'PAINDX2_no_value', 'sample_size' : 'PAINDX2_no_sample'}, inplace = True)

pa03 = pa_base.loc[(pa_base['question_id'] == '_PASTAE2') &
                    (pa_base['response_id'] == 'RESP046')][['location_id', 'year', 'data_value', 'sample_size']]
pa03.rename(columns = {'data_value' : 'PASTAE2_yes_value', 'sample_size' : 'PASTAE2_yes_sample'}, inplace = True)

pa04 = pa_base.loc[(pa_base['question_id'] == '_PASTAE2') &
                    (pa_base['response_id'] == 'RESP054')][['location_id', 'year', 'data_value', 'sample_size']]
pa04.rename(columns = {'data_value' : 'PASTAE2_no_value', 'sample_size' : 'PASTAE2_no_sample'}, inplace = True)

pa05 = pa_base.loc[(pa_base['question_id'] == '_PASTRNG') &
                    (pa_base['response_id'] == 'RESP046')][['location_id', 'year', 'data_value', 'sample_size']]
pa05.rename(columns = {'data_value' : 'PASTRNG_yes_value', 'sample_size' : 'PASTRNG_yes_sample'}, inplace = True)

pa06 = pa_base.loc[(pa_base['question_id'] == '_PASTRNG') &
                    (pa_base['response_id'] == 'RESP054')][['location_id', 'year', 'data_value', 'sample_size']]
pa06.rename(columns = {'data_value' : 'PASTRNG_no_value', 'sample_size' : 'PASTRNG_no_sample'}, inplace = True)

pa07 = pa_base.loc[(pa_base['question_id'] == '_TOTINDA') &
                    (pa_base['response_id'] == 'RESP046')][['location_id', 'year', 'data_value', 'sample_size']]
pa07.rename(columns = {'data_value' : 'TOTINDA_yes_value', 'sample_size' : 'TOTINDA_yes_sample'}, inplace = True)

pa08 = pa_base.loc[(pa_base['question_id'] == '_TOTINDA') &
                    (pa_base['response_id'] == 'RESP054')][['location_id', 'year', 'data_value', 'sample_size']]
pa08.rename(columns = {'data_value' : 'TOTINDA_no_value', 'sample_size' : 'TOTINDA_no_sample'}, inplace = True)

In [41]:
#Merging pa with keys

pa = pd.merge(pa, pa01, how='left', left_on=['location_id', 'year'], right_on=['location_id', 'year'])
pa = pd.merge(pa, pa02, how='left', left_on=['location_id', 'year'], right_on=['location_id', 'year'])
pa = pd.merge(pa, pa03, how='left', left_on=['location_id', 'year'], right_on=['location_id', 'year'])
pa = pd.merge(pa, pa04, how='left', left_on=['location_id', 'year'], right_on=['location_id', 'year'])
pa = pd.merge(pa, pa05, how='left', left_on=['location_id', 'year'], right_on=['location_id', 'year'])
pa = pd.merge(pa, pa06, how='left', left_on=['location_id', 'year'], right_on=['location_id', 'year'])
pa = pd.merge(pa, pa07, how='left', left_on=['location_id', 'year'], right_on=['location_id', 'year'])
pa = pd.merge(pa, pa08, how='left', left_on=['location_id', 'year'], right_on=['location_id', 'year'])

In [42]:
#Checking physical activity dataframe

pa.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 385 entries, 0 to 384
Data columns (total 18 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   location_id         385 non-null    object 
 1   year                385 non-null    int64  
 2   PAINDX2_yes_value   164 non-null    float64
 3   PAINDX2_yes_sample  164 non-null    float64
 4   PAINDX2_no_value    164 non-null    float64
 5   PAINDX2_no_sample   164 non-null    float64
 6   PASTAE2_yes_value   164 non-null    float64
 7   PASTAE2_yes_sample  164 non-null    float64
 8   PASTAE2_no_value    164 non-null    float64
 9   PASTAE2_no_sample   164 non-null    float64
 10  PASTRNG_yes_value   164 non-null    float64
 11  PASTRNG_yes_sample  164 non-null    float64
 12  PASTRNG_no_value    164 non-null    float64
 13  PASTRNG_no_sample   164 non-null    float64
 14  TOTINDA_yes_value   385 non-null    float64
 15  TOTINDA_yes_sample  385 non-null    int64  
 16  TOTINDA_

In [43]:
#Checking physical activity dataframe

pa.head(200)

,location_id,year,PAINDX2_yes_value,PAINDX2_yes_sample,PAINDX2_no_value,PAINDX2_no_sample,PASTAE2_yes_value,PASTAE2_yes_sample,PASTAE2_no_value,PASTAE2_no_sample,PASTRNG_yes_value,PASTRNG_yes_sample,PASTRNG_no_value,PASTRNG_no_sample,TOTINDA_yes_value,TOTINDA_yes_sample,TOTINDA_no_value,TOTINDA_no_sample
0,1,2015,44.6,3142.0,55.4,3883.0,16.7,1067.0,83.3,5889.0,25.8,1644.0,74.2,5543.0,68.1,4851,31.9,2418
1,1,2016,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,70.6,4835,29.4,2178
2,1,2017,42.8,2536.0,57.2,3416.0,16.1,827.0,83.9,5039.0,26.0,1377.0,74.0,4722.0,68.0,4043,32.0,2187
3,1,2018,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,69.3,4497,30.7,2080
4,1,2019,44.8,2885.0,55.2,3431.0,17.6,1041.0,82.4,5110.0,29.2,1759.0,70.8,4680.0,68.5,4483,31.5,2205
5,1,2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,71.2,3771,28.8,1565
6,1,2021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,68.5,3037,31.5,1541
7,10,2015,48.5,1810.0,51.5,1819.0,20.3,681.0,79.7,2898.0,30.5,1032.0,69.5,2674.0,70.6,2626,29.4,1148
8,10,2016,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,73.4,2856,26.6,1197
9,10,2017,46.3,1683.0,53.7,1938.0,19.0,667.0,81.0,2897.0,28.6,1056.0,71.4,2667.0,69.0,2588,31.0,1215


In [44]:
#Checking physical activity dataframe

pa.tail(200)

,location_id,year,PAINDX2_yes_value,PAINDX2_yes_sample,PAINDX2_no_value,PAINDX2_no_sample,PASTAE2_yes_value,PASTAE2_yes_sample,PASTAE2_no_value,PASTAE2_no_sample,PASTRNG_yes_value,PASTRNG_yes_sample,PASTRNG_no_value,PASTRNG_no_sample,TOTINDA_yes_value,TOTINDA_yes_sample,TOTINDA_no_value,TOTINDA_no_sample
185,35,2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,77.0,5371,23.0,1630
186,35,2021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,77.0,4854,23.0,1504
187,36,2015,47.1,5200.0,52.9,5257.0,20.0,2025.0,80.0,8287.0,30.2,3105.0,69.8,7648.0,70.7,7844,29.3,3125
188,36,2016,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,73.7,25150,26.3,8966
189,36,2017,49.3,5280.0,50.7,5174.0,19.8,2029.0,80.2,8267.0,29.9,3120.0,70.1,7618.0,72.8,8059,27.2,2924
190,36,2018,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,76.2,26459,23.8,9211
191,36,2019,50.2,6344.0,49.8,5705.0,24.1,2904.0,75.9,8694.0,37.9,4638.0,62.1,7638.0,72.8,9405,27.2,3545
192,36,2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,74.4,11214,25.6,3530
193,36,2021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,74.2,28913,25.8,10051
194,37,2015,48.1,2960.0,51.9,3039.0,18.9,1088.0,81.1,4861.0,29.3,1689.0,70.7,4517.0,73.8,4607,26.2,1671


# 6. Overall Health

### Overall Health dataset prep

In [45]:
# Making a dataframe of the general health, mental health, diet, and physical activity dataframes

oh = gh.copy()
oh = pd.merge(oh, mh, how='left', left_on=['location_id', 'year'], right_on=['location_id', 'year'])
oh = pd.merge(oh, diet, how='left', left_on=['location_id', 'year'], right_on=['location_id', 'year'])
oh = pd.merge(oh, pa, how='left', left_on=['location_id', 'year'], right_on=['location_id', 'year'])

In [46]:
# Checking Overall Health dataframe

oh.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 385 entries, 0 to 384
Data columns (total 68 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   location_id           385 non-null    object 
 1   year                  385 non-null    int64  
 2   resp039_value         385 non-null    float64
 3   resp039_sample        385 non-null    int64  
 4   resp040_value         385 non-null    float64
 5   resp040_sample        385 non-null    int64  
 6   resp041_value         385 non-null    float64
 7   resp041_sample        385 non-null    int64  
 8   resp042_value         381 non-null    float64
 9   resp042_sample        385 non-null    int64  
 10  resp056_value         385 non-null    float64
 11  resp056_sample        385 non-null    int64  
 12  resp057_value         385 non-null    float64
 13  resp057_sample        385 non-null    int64  
 14  resp058_value         385 non-null    float64
 15  resp058_sample        3

In [47]:
# Checking Overall Health dataframe

oh.head(100)

,location_id,year,resp039_value,resp039_sample,resp040_value,resp040_sample,resp041_value,resp041_sample,resp042_value,resp042_sample,...,PASTAE2_no_value,PASTAE2_no_sample,PASTRNG_yes_value,PASTRNG_yes_sample,PASTRNG_no_value,PASTRNG_no_sample,TOTINDA_yes_value,TOTINDA_yes_sample,TOTINDA_no_value,TOTINDA_no_sample
0,1,2015,35.6,2639,33.0,2523,28.9,2075,2.4,146,...,83.3,5889.0,25.8,1644.0,74.2,5543.0,68.1,4851,31.9,2418
1,1,2016,35.7,2352,33.8,2259,28.4,1805,2.1,110,...,NaN,NaN,NaN,NaN,NaN,NaN,70.6,4835,29.4,2178
2,1,2017,36.3,2260,33.9,2152,28.5,1748,1.3,85,...,83.9,5039.0,26.0,1377.0,74.0,4722.0,68.0,4043,32.0,2187
3,1,2018,36.2,2238,33.4,2169,28.4,1633,2.0,104,...,NaN,NaN,NaN,NaN,NaN,NaN,69.3,4497,30.7,2080
4,1,2019,36.1,2338,34.2,2295,27.5,1758,2.2,130,...,82.4,5110.0,29.2,1759.0,70.8,4680.0,68.5,4483,31.5,2205
5,1,2020,39.0,1918,33.7,1695,25.4,1289,1.8,71,...,NaN,NaN,NaN,NaN,NaN,NaN,71.2,3771,28.8,1565
6,1,2021,39.9,1652,31.9,1412,26.4,1112,1.8,71,...,NaN,NaN,NaN,NaN,NaN,NaN,68.5,3037,31.5,1541
7,10,2015,29.7,1146,37.2,1391,31.7,1092,1.4,59,...,79.7,2898.0,30.5,1032.0,69.5,2674.0,70.6,2626,29.4,1148
8,10,2016,30.7,1180,37.3,1407,30.2,1063,1.8,52,...,NaN,NaN,NaN,NaN,NaN,NaN,73.4,2856,26.6,1197
9,10,2017,31.8,1224,36.7,1329,29.5,1039,2.0,56,...,81.0,2897.0,28.6,1056.0,71.4,2667.0,69.0,2588,31.0,1215


# 7. Exports

In [17]:
#Exporting dataset

oh.to_pickle(os.path.join(path, '02 Data','Prepared Data', 'BRFSS Overall Health.pkl'))
oh.to_csv(os.path.join(path, '02 Data','Prepared Data', 'BRFSS Overall Health.csv'))

gh.to_pickle(os.path.join(path, '02 Data','Prepared Data', 'BRFSS General Health.pkl'))
gh.to_csv(os.path.join(path, '02 Data','Prepared Data', 'BRFSS General Health.csv'))

mh.to_pickle(os.path.join(path, '02 Data','Prepared Data', 'BRFSS Mental Health.pkl'))
mh.to_csv(os.path.join(path, '02 Data','Prepared Data', 'BRFSS Mental Health.csv'))

diet.to_pickle(os.path.join(path, '02 Data','Prepared Data', 'BRFSS Dietary Health.pkl'))
diet.to_csv(os.path.join(path, '02 Data','Prepared Data', 'BRFSS Dietary Health.csv'))

pa.to_pickle(os.path.join(path, '02 Data','Prepared Data', 'BRFSS Physical Acticity.pkl'))
pa.to_csv(os.path.join(path, '02 Data','Prepared Data', 'BRFSS Physical Acticity.csv'))